In [25]:
import pandas as pd
import numpy as np
import pandas_datareader as pdr
import pandas_datareader.data as web
import matplotlib.pyplot as plt
%matplotlib inline  
from datetime import datetime
import pytest
from pypfopt import black_litterman, risk_models

symbols = ['SBUX', 'GOOG', 'META', 'AAPL', 'BAC', 'JPM', 'T', 'GE', 'MSFT', 'XOM']
 
all_stocks = pd.DataFrame()


In [26]:
from pypfopt import black_litterman
from pypfopt.black_litterman import BlackLittermanModel
from pypfopt import risk_models, expected_returns

In [29]:
def get_data():
    return pd.read_csv(resource("stock_prices.csv"), parse_dates=True, index_col="date")

In [30]:
def get_market_caps():
    mcaps = {
        "GOOG": 927e9,
        "AAPL": 1.19e12,
        "META": 574e9,
        "BABA": 533e9,
        "AMZN": 867e9,
        "GE": 96e9,
        "AMD": 43e9,
        "WMT": 339e9,
        "BAC": 301e9,
        "GM": 51e9,
        "T": 61e9,
        "UAA": 78e9,
        "SHLD": 0,
        "XOM": 295e9,
        "RRC": 1e9,
        "BBY": 22e9,
        "MA": 288e9,
        "PFE": 212e9,
        "JPM": 422e9,
        "SBUX": 102e9,
    }
    return mcaps

In [31]:
def get_data_by_symbols(symbols,data_source,ohlc,begin_date=None,end_date=None):
    out = []
    new_symbols = []
    
    for symbol in symbols:
        df = web.DataReader(symbol, data_source, begin_date, end_date)\
        [['High','Low','Open','Close','Volume','Adj Close']]
        new_symbols.append(symbol) 
        out.append(df[ohlc].astype('float'))
        data = pd.concat(out, axis = 1)
        data.columns = new_symbols
        
    return data

In [32]:
start = pd.Timestamp('2015-01')
end = pd.Timestamp('2020-12')
data_source = 'yahoo'

In [34]:
df_prices = get_data_by_symbols(symbols,data_source=data_source,ohlc='Adj Close', begin_date=start, end_date=end)

In [35]:
df_prices

,SBUX,GOOG,META,AAPL,BAC,JPM,T,GE,MSFT,XOM
Date,,,,,,,,,,
2015-01-02,35.324505,26.168653,78.449997,24.644018,15.456273,50.305370,13.803200,167.345612,40.926441,64.727486
2015-01-05,34.647850,25.623152,77.190002,23.949759,15.007266,48.743645,13.672790,164.273819,40.550083,62.956425
2015-01-06,34.365925,25.029282,76.150002,23.952015,14.558257,47.479755,13.693163,160.734543,39.954926,62.621716
2015-01-07,35.211735,24.986401,76.150002,24.287870,14.627335,47.552216,13.773004,160.801346,40.462559,63.256241
2015-01-08,35.779938,25.065184,78.180000,25.221067,14.929551,48.614841,13.910028,162.737900,41.652901,64.309128
...,...,...,...,...,...,...,...,...,...,...
2020-11-24,94.999428,88.444000,276.920013,113.992645,27.775000,116.618179,18.676151,82.983536,210.705597,38.345352
2020-11-25,94.902786,88.571503,275.589996,114.843849,27.822927,115.398285,18.522806,83.380585,210.715454,37.276649
2020-11-27,95.347351,89.659500,277.809998,115.398125,27.784588,114.632317,18.548363,82.586494,212.055405,36.710327


In [46]:
mu = expected_returns.mean_historical_return(df_prices)
mu

SBUX    0.183318
GOOG    0.232292
META    0.245135
AAPL    0.309907
BAC     0.102405
JPM     0.147185
T       0.050296
GE     -0.116301
MSFT    0.322049
XOM    -0.098100
dtype: float64

In [47]:
cov_matrix = risk_models.sample_cov(df_prices)
cov_matrix

,SBUX,GOOG,META,AAPL,BAC,JPM,T,GE,MSFT,XOM
SBUX,0.070171,0.039107,0.039958,0.038726,0.049672,0.044988,0.025763,0.041688,0.042900,0.035225
GOOG,0.039107,0.073183,0.057960,0.048820,0.043363,0.038345,0.022746,0.034876,0.055314,0.031012
META,0.039958,0.057960,0.102231,0.054850,0.042073,0.036577,0.019195,0.034930,0.054591,0.029165
AAPL,0.038726,0.048820,0.054850,0.088001,0.047181,0.041815,0.025187,0.037993,0.056972,0.033309
BAC,0.049672,0.043363,0.042073,0.047181,0.113289,0.091520,0.039148,0.070032,0.048616,0.059768
JPM,0.044988,0.038345,0.036577,0.041815,0.091520,0.086632,0.036288,0.063068,0.043720,0.054451
T,0.025763,0.022746,0.019195,0.025187,0.039148,0.036288,0.048101,0.036229,0.025294,0.031896
GE,0.041688,0.034876,0.034930,0.037993,0.070032,0.063068,0.036229,0.141631,0.037550,0.056092
MSFT,0.042900,0.055314,0.054591,0.056972,0.048616,0.043720,0.025294,0.037550,0.078146,0.032937
XOM,0.035225,0.031012,0.029165,0.033309,0.059768,0.054451,0.031896,0.056092,0.032937,0.076466


In [49]:
from pypfopt.efficient_frontier import EfficientFrontier

In [50]:
# Optimize for maximal Sharpe ratio
ef = EfficientFrontier(mu, cov_matrix)
weights = ef.max_sharpe()
ef.portfolio_performance(verbose=True)

Expected annual return: 31.8%
Annual volatility: 26.4%
Sharpe Ratio: 1.13


(0.31778475745043283, 0.26403143169275256, 1.1278382863028151)

In [40]:
views = pd.Series(0.1, index=S.columns)

In [ ]:
#SBUX will drop 20% (absolute)
#MSFT will rise by 5% (absolute)
#GOOG outperforms FB by 10%
#BAC and JPM will outperform T and GE by 15%

In [51]:
Q = np.array([-0.20, 0.05, 0.10, 0.15]).reshape(-1, 1)
Q

array([[-0.2 ],
       [ 0.05],
       [ 0.1 ],
       [ 0.15]])

In [ ]:
# -Each view has a corresponding row in the picking matrix (the order matters)
# -Absolute views have a single 1 in the column corresponding to the ticker’s order in the universe.
# -Relative views have a positive number in the nominally outperforming asset columns and a negative 
#    number in the nominally underperforming asset columns. The numbers in each row should sum up to 0.

In [55]:
P = np.array(
    [
        [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
        [0, 1, -1, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0.5, 0.5, -0.5, -0.5, 0, 0],
    ]
)

In [56]:
viewdict = {"AAPL": 0.20, "JPM": -0.30, "AAPL": 0, "SBUX": -0.2, "T": 0.15}
bl = BlackLittermanModel(cov_matrix, absolute_views=viewdict)
bl

In [62]:
bl.bl_weights()

OrderedDict([('SBUX', 3.4745949497953776),
             ('GOOG', 3.716561955589356e-16),
             ('META', -4.0360384811253155e-17),
             ('AAPL', -1.0445362556693438),
             ('BAC', 6.980883177815883e-15),
             ('JPM', 5.130091434908741),
             ('T', -6.560150129034779),
             ('GE', 1.086760381781198e-17),
             ('MSFT', -1.109228691471946e-15),
             ('XOM', -2.3060082663028204e-15)])

In [71]:
rets = bl.bl_returns()
es = EfficientFrontier(rets, cov_matrix)

In [72]:
rets

SBUX   -0.104192
GOOG   -0.052022
META   -0.056297
AAPL   -0.036120
BAC    -0.132033
JPM    -0.125359
T       0.026005
GE     -0.075072
MSFT   -0.058120
XOM    -0.061968
dtype: float64

In [73]:
es.efficient_return(0.01)

OrderedDict([('SBUX', 0.0),
             ('GOOG', 0.0807809560850821),
             ('META', 0.0671716885291813),
             ('AAPL', 0.0671766193549566),
             ('BAC', 0.0),
             ('JPM', 0.0),
             ('T', 0.7848707360307801),
             ('GE', 0.0),
             ('MSFT', 0.0),
             ('XOM', 0.0)])

In [74]:
# We can use the same helper methods as before
weights = es.clean_weights()
print(weights)
es.portfolio_performance(verbose=True)

OrderedDict([('SBUX', 0.0), ('GOOG', 0.08078), ('META', 0.06717), ('AAPL', 0.06718), ('BAC', 0.0), ('JPM', 0.0), ('T', 0.78487), ('GE', 0.0), ('MSFT', 0.0), ('XOM', 0.0)])
Expected annual return: 1.0%
Annual volatility: 20.0%
Sharpe Ratio: -0.05


(0.01, 0.20046206337448005, -0.04988475041943052)